In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

biased_file_path = "/content/drive/MyDrive/bias_data/WNC/biased_full.txt"

biased_df = pd.read_csv(
    biased_file_path,
    sep="\t",
    header=None,
    names=["id", "src_tok", "tgt_tok", "src_raw", "tgt_raw", "src_POS_tags", "tgt_parse_tags"],
    on_bad_lines="skip"
)

In [3]:
biased_df.head()

,id,src_tok,tgt_tok,src_raw,tgt_raw,src_POS_tags,tgt_parse_tags
0,258378316,"during the campaign , controversy erupted over...","during the campaign , some pointed out alleged...","during the campaign, controversy erupted over ...","during the campaign, some pointed out alleged ...",ADP DET NOUN PUNCT NOUN VERB ADP VERB NOUN ADP...,prep det pobj punct nsubj ROOT prep amod pobj ...
1,486527143,nic ##aea was con ##vo ##ked by the emperor co...,nic ##aea was con ##vo ##ked by the emperor co...,nicaea was convoked by the emperor constantine...,nicaea was convoked by the emperor constantine...,NOUN NOUN VERB VERB VERB VERB ADP DET NOUN NOU...,nsubjpass nsubjpass auxpass ROOT ROOT ROOT age...
2,54024499,it was rather unfortunate that he ve ##hem ##e...,he ve ##hem ##ently opposed the bud ##ding ind...,it was rather unfortunate that he vehemently o...,he vehemently opposed the budding indian scien...,PRON VERB ADV ADJ ADP PRON ADV ADV ADV VERB DE...,nsubj ROOT advmod acomp mark nsubj advmod advm...
3,160186886,dennis the menace is an american animated seri...,dennis the menace is an american animated seri...,dennis the menace is an american animated seri...,dennis the menace is an american animated seri...,VERB DET NOUN VERB DET ADJ VERB NOUN VERB ADP ...,csubj det dobj ROOT det amod amod attr acl age...
4,8797183,"today , on large farms , motorcycles , dogs or...","today , on large farms , motorcycles , dogs or...","today, on large farms, motorcycles, dogs or me...","today, on large farms, motorcycles, dogs or pe...",NOUN PUNCT ADP ADJ NOUN PUNCT NOUN PUNCT NOUN ...,npadvmod punct prep amod pobj punct conj punct...


In [4]:
sample_size = 5
examples = biased_df.sample(sample_size, random_state=45)[['src_raw', 'tgt_raw']]

# examples of src_raw and tgt_raw
for i, row in examples.iterrows():
    print(f"Example {i+1}:")
    print(f"Biased Sentence (src_raw): {row['src_raw']}")
    print(f"Neutralized Sentence (tgt_raw): {row['tgt_raw']}")
    print("-" * 50)

Example 62417:
Biased Sentence (src_raw): unfortunately, after the war, as the nation expanded so did the practice of lynching.
Neutralized Sentence (tgt_raw): after the war, as the nation expanded so did the practice of lynching.
--------------------------------------------------
Example 160048:
Biased Sentence (src_raw): unfortunately, the highly elliptical orbit of the sirius satellite constellation can pose difficulties for the reliable delivery of the signal to stationary antennas in certain parts of the country.
Neutralized Sentence (tgt_raw): the highly elliptical orbit of the sirius satellite constellation can pose difficulties for the reliable delivery of the signal to stationary antennas in certain parts of the country.
--------------------------------------------------
Example 26192:
Biased Sentence (src_raw): the medes kurds were an ancient iranian people who lived in the northwestern portions of present-day iran, and roughly the areas of present day kurdistan, hamedan, teh

In [5]:
biased_df.shape

(181473, 7)

Since we are using Google Colab with limited compute resources, we reduce the size of this dataset to 20,000 random rows and then choose 2,000 rows with the least similarities/most edits between source and target, so that the model learns the most.

In [6]:
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

sampled_df = biased_df.sample(n=20000, random_state=42)

vectorizer = TfidfVectorizer()
src_vectors = vectorizer.fit_transform(sampled_df['src_raw'])
tgt_vectors = vectorizer.transform(sampled_df['tgt_raw'])

# cosine similarity between src_raw and tgt_raw for each pair
similarities = cosine_similarity(src_vectors, tgt_vectors).diagonal()
sampled_df['similarity'] = similarities

lowest_similarity_df = sampled_df.sort_values(by='similarity', ascending=True).head(2000)

In [7]:
lowest_similarity_df.shape

(2000, 8)

In [8]:
lowest_similarity_df.head()

,id,src_tok,tgt_tok,src_raw,tgt_raw,src_POS_tags,tgt_parse_tags,similarity
77551,125281387,2000 - 2005 : attempts at rebuilding reputation,2000 - 2005 : return to the playoffs,2000-2005: attempts at rebuilding reputation,2000-2005: return to the playoffs,NUM SYM NUM PUNCT NOUN ADP VERB NOUN,nummod punct prep punct ROOT prep pcomp dobj,0.296064
7259,244059551,"however , to date , the governor has not fulfi...","however , to date , the governor has not recog...","however, to date, the governor has not fulfill...","however, to date, the governor has not recogni...",ADV PUNCT ADP NOUN PUNCT DET NOUN VERB ADV VER...,advmod punct prep pobj punct det nsubj aux neg...,0.301056
52849,271548246,"should the "" build the web "" guide ##line be r...","review of the merge of "" build the web "" with ...","should the ""build the web"" guideline be reinst...","review of the merge of ""build the web"" with mo...",VERB DET PUNCT VERB DET NOUN PUNCT NOUN NOUN V...,aux det punct ROOT det nmod punct nsubjpass ns...,0.322793
160267,289164642,"one of the game ' s developer ' s , ta ##sos ,...","one of the game developers , ta ##sos , has of...","one of the game 's developer's , tasos, has st...","one of the game developers , tasos, has offere...",NUM ADP DET NOUN PUNCT VERB NOUN PUNCT NOUN PU...,nsubj prep det poss case compound pobj punct c...,0.324562
127903,3557312,42 below is the most southern made vodka in th...,the uk magazine class rated it the number two ...,42 below is the most southern made vodka in th...,the uk magazine class rated it the number two ...,NUM ADV VERB DET ADV ADJ VERB NOUN ADP DET NOU...,nsubj advmod ROOT det advmod attr acl oprd pre...,0.325155


## Structure the dataset as prompts and responses for PEFT QLORA fine-tuning.

In [9]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
import pandas as pd
from datasets import Dataset

def prepare_peft_dataset(df):
    """
    Prepare dataset for PEFT QLoRA fine-tuning.

    Args:
        df (pd.DataFrame): DataFrame with columns `src_raw` (biased sentences) and `tgt_raw` (neutral sentences).

    Returns:
        Dataset: HuggingFace Dataset with `prompt` and `response` fields.
    """
    df["prompt"] = df["src_raw"].apply(
        lambda x: (
            f"Rewrite the following sentence to be neutral and unbiased, don't explain what changes were made and why:\n\n{x}\n\nRewritten sentence:"
        )
    )

    df["response"] = df["tgt_raw"].apply(
        lambda x: (
            f"Rewritten sentence:\n\n{x}"
        )
    )

    df = df.dropna(subset=["prompt", "response"])

    return Dataset.from_pandas(df[["prompt", "response"]])

peft_dataset = prepare_peft_dataset(lowest_similarity_df)

split_dataset = peft_dataset.train_test_split(test_size=0.1, seed=40)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

print("Sample Prompt:")
print(train_dataset[0]["prompt"])
print("\nSample Response:")
print(train_dataset[0]["response"])

Sample Prompt:
Rewrite the following sentence to be neutral and unbiased, don't explain what changes were made and why:

however, there are less well-documented reports that he ate meat occasionally during the 1930s.

Rewritten sentence:

Sample Response:
Rewritten sentence:

however, there are largely anecdotal reports that he ate meat occasionally during the 1930s.


In [11]:
from huggingface_hub import login
login(token='') # huggingface personal token here

In [12]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.5 MB/s eta 0:00:00


In [13]:
pip install accelerate==0.26.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, TaskType, get_peft_model
from transformers import TrainingArguments, Trainer

model_name = "meta-llama/Llama-2-7b-hf"

use_4bit = True
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none"
)

model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [15]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="no",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_32bit",
    report_to="none",
    remove_unused_columns=False
)

def data_collator(batch):
    inputs = tokenizer(
        [example["prompt"] for example in batch],
        text_target=[example["response"] for example in batch],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs["labels"] = inputs["input_ids"].clone()
    return inputs

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

<ipython-input-15-07ba4f0c4f2a>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
10,2.669700
20,1.658900
30,1.335000
40,1.333500
50,1.294700
60,1.329300
70,1.327700
80,1.342800
90,1.304500
100,1.322000


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

Step,Training Loss
10,2.669700
20,1.658900
30,1.335000
40,1.333500
50,1.294700
60,1.329300
70,1.327700
80,1.342800
90,1.304500
100,1.322000


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=336, training_loss=1.3407952047529674, metrics={'train_runtime': 2632.7519, 'train_samples_per_second': 2.051, 'train_steps_per_second': 0.128, 'total_flos': 1.3991860914536448e+16, 'train_loss': 1.3407952047529674, 'epoch': 2.986666666666667})

In [16]:
model.save_pretrained("/content/drive/MyDrive/qlora_debiased_model")
tokenizer.save_pretrained("/content/drive/MyDrive/qlora_debiased_model")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('/content/drive/MyDrive/qlora_debiased_model/tokenizer_config.json',
 '/content/drive/MyDrive/qlora_debiased_model/special_tokens_map.json',
 '/content/drive/MyDrive/qlora_debiased_model/tokenizer.model',
 '/content/drive/MyDrive/qlora_debiased_model/added_tokens.json',
 '/content/drive/MyDrive/qlora_debiased_model/tokenizer.json')